In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
from sklearn.metrics import ConfusionMatrixDisplay
import xgboost as xgb

In [3]:
import os
os.chdir("D:\\elliptic\\Elliptic_Dataset")
print("\nTransaction features: \n")
df_txs_features = pd.read_csv("txs_features.csv")
print(df_txs_features)

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv("txs_classes.csv")
print(df_txs_classes)

print("\nTransaction-Transaction edgelist: \n")
df_txs_edgelist = pd.read_csv("txs_edgelist.csv")

print(df_txs_edgelist)


Transaction features: 

             txId  Time step  Local_feature_1  Local_feature_2  \
0            3321          1        -0.169615        -0.184668   
1           11108          1        -0.137586        -0.184668   
2           51816          1        -0.170103        -0.184668   
3           68869          1        -0.114267        -0.184668   
4           89273          1         5.202107        -0.210553   
...           ...        ...              ...              ...   
203764  158304003         49        -0.165622        -0.139563   
203765  158303998         49        -0.167040        -0.139563   
203766  158303966         49        -0.167040        -0.139563   
203767  161526077         49        -0.172212        -0.139573   
203768  194103537         49        -0.172212        -0.139573   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
0             -1.201369        -0.121970        -0.043875        -0.113002   
1             -1.201369   

In [4]:
print("\ntxs_features.csv for txId = 272145560\n")
print(df_txs_features[df_txs_features['txId']==272145560])

print("\ntxs_classes.csv for txId = 272145560\n")
print(df_txs_classes[df_txs_classes['txId']==272145560])

print("\ntxs_edgelist.csv for txId = 272145560\n")
df_txs_edgelist[(df_txs_edgelist['txId1']==272145560) | (df_txs_edgelist['txId2']==272145560)]


txs_features.csv for txId = 272145560

             txId  Time step  Local_feature_1  Local_feature_2  \
105573  272145560         24        -0.155493        -0.107012   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
105573        -1.201369         -0.12197        -0.043875        -0.113002   

        Local_feature_7  Local_feature_8  ...  in_BTC_min  in_BTC_max  \
105573        -0.061584        -0.145749  ...      2.7732      2.7732   

        in_BTC_mean  in_BTC_median  in_BTC_total  out_BTC_min  out_BTC_max  \
105573       2.7732         2.7732        2.7732     0.001917     2.770883   

        out_BTC_mean  out_BTC_median  out_BTC_total  
105573        1.3864          1.3864         2.7728  

[1 rows x 184 columns]

txs_classes.csv for txId = 272145560

             txId  class
105573  272145560      1

txs_edgelist.csv for txId = 272145560



,txId1,txId2
123072,272145560,296926618
123272,272145560,272145556
125873,299475624,272145560


In [5]:
df_txs_features = df_txs_features.merge(df_txs_classes, left_on = "txId" , right_on = "txId" , how = "left")
# print(txs.shape)

print(df_txs_features["class"].value_counts())

class
3    157205
2     42019
1      4545
Name: count, dtype: int64


In [6]:
df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [7]:
for column in df_txs_features.columns[168:]:
    if column != "class":
        feature = np.array(df_txs_features[column]).reshape(-1,1)
        scaler = MinMaxScaler()
        scaler.fit(feature)
        feature_scaled = scaler.transform(feature)
        df_txs_features[column] = feature_scaled.reshape(1,-1)[0]

In [8]:
# remove 'unknown' transactions
data = df_txs_features["txId"] #.loc[(df_txs_features['class'] != 3), 'txId']
df_txs_features_selected = df_txs_features.loc[df_txs_features['txId'].isin(data)]
print(df_txs_features_selected["class"].value_counts())

class
3    156759
2     41500
1      4545
Name: count, dtype: int64


In [9]:
df_txs_features_selected

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.000047,0.000047,0.000047,0.000047,8.301504e-05,0.000032,0.000089,8.904096e-05,0.000047,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,0.000493,0.000493,0.000493,0.000493,2.915711e-04,0.000444,0.000936,9.357923e-04,0.000493,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.000040,0.000040,0.000040,0.000040,1.134016e-04,0.000020,0.000076,7.612341e-05,0.000040,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,0.000702,0.000272,0.000088,0.000817,6.113009e-04,0.000714,0.001552,1.552291e-03,0.000817,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,0.074805,0.074805,0.074805,0.074805,6.466160e-11,0.003648,0.000022,1.451405e-07,0.074805,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,0.010179,0.010179,0.010179,0.010179,8.223464e-04,0.010104,0.019336,1.933576e-02,0.010179,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,0.010029,0.010029,0.010029,0.010029,1.012352e-05,0.010098,0.019052,1.905181e-02,0.010029,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,0.009973,0.009973,0.009973,0.009973,4.604647e-04,0.009961,0.018945,1.894453e-02,0.009973,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,0.009891,0.009891,0.009891,0.009891,1.505606e-04,0.009935,0.018790,1.879015e-02,0.009891,3


In [10]:
import torch
from torch_geometric.data import Data
# from torch_geometric.utils import to_undirected  # nếu muốn graph vô hướng thì mở comment

# ========= 0. Chuẩn hoá kiểu dữ liệu txId / edgelist =========
df_txs_features_selected = df_txs_features_selected.copy()
df_txs_edgelist          = df_txs_edgelist.copy()

df_txs_features_selected['txId'] = df_txs_features_selected['txId'].astype(int)
df_txs_edgelist['txId1']         = df_txs_edgelist['txId1'].astype(int)
df_txs_edgelist['txId2']         = df_txs_edgelist['txId2'].astype(int)

# ========= 1. Chia node train / test + tạo label =========
# chỉ dùng class 1,2 ; bỏ class 3
train_nodes = df_txs_features_selected[
    (df_txs_features_selected['Time step'] < 35) &
    (df_txs_features_selected['class'] != 3)
].copy()

test_nodes = df_txs_features_selected[
    (df_txs_features_selected['Time step'] >= 35) &
    (df_txs_features_selected['class'] != 3)
].copy()

# map class -> label: 1 -> 0, 2 -> 1
label_map = {1: 0, 2: 1}
train_nodes['label'] = train_nodes['class'].map(label_map)
test_nodes['label']  = test_nodes['class'].map(label_map)

# ========= 2. Chuẩn bị feature tensor =========
feature_cols = [
    c for c in df_txs_features_selected.columns
    if c not in ['txId', 'class', 'Time step']
]

X_train_gcn = torch.tensor(train_nodes[feature_cols].values, dtype=torch.float)
y_train_gcn = torch.tensor(train_nodes['label'].values, dtype=torch.long)

X_test_gcn = torch.tensor(test_nodes[feature_cols].values, dtype=torch.float)
y_test_gcn = torch.tensor(test_nodes['label'].values, dtype=torch.long)

# ========= 3. Hàm build_edge_index cho một tập node =========
def build_edge_index(df_edges, valid_txids):
    """
    df_edges    : dataframe có cột txId1, txId2
    valid_txids : array/list các txId được giữ lại (cùng kiểu int)
    Trả về:
        - edge_index: LongTensor [2, num_edges]
        - id2idx    : dict txId -> node_index (index trong X)
    """
    import numpy as np

    node_ids = np.asarray(valid_txids, dtype=int)
    id2idx = {tid: i for i, tid in enumerate(node_ids)}

    # lọc cạnh chỉ chứa node thuộc tập valid
    mask = (
        df_edges['txId1'].isin(node_ids) &
        df_edges['txId2'].isin(node_ids)
    )
    edges_df = df_edges.loc[mask, ['txId1', 'txId2']]

    if len(edges_df) == 0:
        edge_index = torch.empty((2, 0), dtype=torch.long)
        return edge_index, id2idx

    # map txId -> index
    edges_idx = edges_df.replace(id2idx).values  # shape [num_edges, 2]

    edge_index = torch.tensor(edges_idx.T, dtype=torch.long)  # [2, num_edges]

    # nếu muốn graph vô hướng thì dùng:
    # edge_index = to_undirected(edge_index)

    return edge_index, id2idx

# build edge cho train / test
train_txids_np = train_nodes['txId'].astype(int).values
test_txids_np  = test_nodes['txId'].astype(int).values

edge_index_train, id2idx_train = build_edge_index(df_txs_edgelist, train_txids_np)
edge_index_test,  id2idx_test  = build_edge_index(df_txs_edgelist, test_txids_np)

# ========= 4. Tạo PyG Data object =========
train_txids = torch.tensor(train_txids_np, dtype=torch.long)
test_txids  = torch.tensor(test_txids_np,  dtype=torch.long)

train_data = Data(
    x=X_train_gcn,
    edge_index=edge_index_train,
    y=y_train_gcn,
)
test_data = Data(
    x=X_test_gcn,
    edge_index=edge_index_test,
    y=y_test_gcn,
)

# lưu lại txId theo đúng thứ tự node
train_data.node_ids = train_txids
test_data.node_ids  = test_txids


d:\elliptic\venv1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [11]:
print(train_data)
print(test_data)

Data(x=[29699, 182], edge_index=[2, 22796], y=[29699], node_ids=[29699])
Data(x=[16346, 182], edge_index=[2, 13616], y=[16346], node_ids=[16346])


In [12]:
print("Check NaN in train features:", torch.isnan(train_data.x).any().item())
print("Check Inf in train features:", torch.isinf(train_data.x).any().item())
print("Check NaN in train labels:", torch.isnan(train_data.y.float()).any().item())

Check NaN in train features: False
Check Inf in train features: False
Check NaN in train labels: False


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.metrics import precision_recall_fscore_support, f1_score, confusion_matrix

# ======================
# 1. Định nghĩa GCN model
# ======================
class GCN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hid_dim)
        self.conv2 = GCNConv(hid_dim, out_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [14]:
from torch_geometric.nn import GCN2Conv

class DeepGCNII(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, num_layers=16, alpha=0.1):
        super().__init__()
        self.lin_in = nn.Linear(in_dim, hid_dim)
        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCN2Conv(hid_dim, alpha=alpha, theta=None, layer=None))
        self.lin_out = nn.Linear(hid_dim, out_dim)

    def forward(self, x, edge_index):
        x0 = self.lin_in(x)
        x  = x0
        for layer, conv in enumerate(self.convs):
            x = conv(x, x0, edge_index)   # GCNII uses (x, x0)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        out = self.lin_out(x)
        return out


In [15]:
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hid_dim)
        self.conv2 = SAGEConv(hid_dim, out_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [16]:
device = torch.device("xpu" if torch.xpu.is_available() else "cpu")
# device = torch.device("cpu")
device

device(type='xpu')

In [17]:
class_sample_count = torch.bincount(y_train_gcn, minlength=2).float()
print("Class distribution:", class_sample_count.tolist())

eps = 1e-8
weights = 1.0 / (class_sample_count + eps)  # tensor([w0, w1])



Class distribution: [3462.0, 26237.0]


In [18]:
from torch.optim.lr_scheduler import StepLR
in_dim = train_data.x.size(1)
model = DeepGCNII(in_dim=in_dim, hid_dim=64, out_dim=2).to(device)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# scheduler = StepLR(opt, step_size=100, gamma=0.5)
crit = nn.CrossEntropyLoss(weight=weights.to(device))

# ======================
# 3. Training loop
# ======================
def train_epoch(data):
    model.train()
    opt.zero_grad()
    out = model(data.x.to(device), data.edge_index.to(device))
    loss = crit(out, data.y.to(device))
    loss.backward()
    opt.step()
    return loss.item()

for epoch in range(1, 801):
    loss = train_epoch(train_data)
    if epoch % 50 == 0:
        print(f"[Epoch {epoch}] loss={loss:.4f}")
    # scheduler.step()



[Epoch 50] loss=0.1421
[Epoch 100] loss=0.0978
[Epoch 150] loss=0.0822
[Epoch 200] loss=0.0686
[Epoch 250] loss=0.0698
[Epoch 300] loss=0.0709
[Epoch 350] loss=0.0596
[Epoch 400] loss=0.0597
[Epoch 450] loss=0.0557
[Epoch 500] loss=0.0551
[Epoch 550] loss=0.0840
[Epoch 600] loss=0.0523
[Epoch 650] loss=0.0513
[Epoch 700] loss=0.0584
[Epoch 750] loss=0.0547
[Epoch 800] loss=0.0597


In [20]:
# ======================
# 4. Evaluation
# ======================
@torch.no_grad()
def evaluate(data, name="TEST"):
    model.eval()
    out = model(data.x.to(device), data.edge_index.to(device))
    preds = out.argmax(dim=1).cpu().numpy()
    y_true = data.y.cpu().numpy()

    prec, rec, f1, _ = precision_recall_fscore_support(y_true, preds, zero_division=0)
    micro_f1 = f1_score(y_true, preds, average='micro', zero_division=0)
    macro_f1 = f1_score(y_true, preds, average='macro', zero_division=0)
    cm = confusion_matrix(y_true, preds, labels=[0,1])

    print(f"\n{name} Evaluation")
    print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (prec[0], rec[0], f1[0]))
    print("Micro-F1: %.3f" % micro_f1)
    print("Macro-F1: %.3f" % macro_f1)
    print("Confusion Matrix:\n", cm)

# Evaluate cả train và test
evaluate(train_data, "TRAIN")
evaluate(test_data, "TEST")


TRAIN Evaluation
Precision: 0.896 
Recall: 0.994 
F1 Score: 0.943
Micro-F1: 0.986
Macro-F1: 0.967
Confusion Matrix:
 [[ 3441    21]
 [  398 25839]]

TEST Evaluation
Precision: 0.633 
Recall: 0.627 
F1 Score: 0.630
Micro-F1: 0.951
Macro-F1: 0.802
Confusion Matrix:
 [[  679   404]
 [  393 14870]]
